# Prepare environment

In [1]:
!pip install git+https://github.com/katarinagresova/ensembl_scraper.git@6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd

  Cloning https://github.com/katarinagresova/ensembl_scraper.git (to revision 6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd) to /tmp/pip-req-build-balwl2li
  Running command git clone --filter=blob:none -q https://github.com/katarinagresova/ensembl_scraper.git /tmp/pip-req-build-balwl2li
  Running command git rev-parse -q --verify 'sha^6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd'
  Running command git fetch -q https://github.com/katarinagresova/ensembl_scraper.git 6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd
  Resolved https://github.com/katarinagresova/ensembl_scraper.git to commit 6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 271 kB 1.0 MB/s            
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
     |████████████████████████████████| 865 kB 827 kB/s            
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached scipy-1.8.0-cp38-cp38-manylinu

# Create config file

In [2]:
import yaml

config = {
    "root_dir": "../../datasets/",
    "organisms": {
        "homo_sapiens": {
            "regulatory_feature"
        }
    }
}

user_config = 'user_config.yaml'
with open(user_config, 'w') as handle:
  yaml.dump(config, handle)

# Prepare directories

In [3]:
from pathlib import Path

BASE_FILE_PATH = Path("../../datasets/human_ocr_ensembl/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)

# Run tool

In [4]:
!python -m scraper.ensembl_scraper -c user_config.yaml

Processing feature files:   0%|                           | 0/1 [00:00<?, ?it/s]INFO:root:download_file(): Going to download file from path ftp://ftp.ensembl.org/pub/release-100/mysql/regulation_mart_100/hsapiens_regulatory_feature__regulatory_feature__main.txt.gz
INFO:root:download_file(): File downloaded to path ../../datasets//tmp//homo_sapiens_regulatory_feature.txt.gz.
INFO:root:parse_feature_file(): Going to parse file ../../datasets//tmp//homo_sapiens_regulatory_feature.txt.gz
INFO:root:parse_feature_file(): Done parsing file ../../datasets//tmp//homo_sapiens_regulatory_feature.txt.gz


Processing feature types:   0%|                           | 0/6 [00:00<?, ?it/s]INFO:root:find_sequences(): Going to find sequences based on genomic loci.
INFO:root:download_2bit_file(): Going to download 2bit file hg38
INFO:root:download_2bit_file(): File for hg38 downloaded to path ../../datasets//tmp/hg38.2bit.
INFO:root:find_sequences(): Done finding sequences.
INFO:root:remove_low_quality():

# Cleaning

In [5]:
!mv ../../datasets/homo_sapiens_regulatory_feature_open_chromatin_region ../../datasets/human_ocr_ensembl/

In [6]:
!rm user_config.yaml

In [8]:
!rm -rf ../../datasets/tmp/

In [12]:
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_CTCF_binding_site
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_enhancer
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_promoter
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_promoter_flanking_region
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_TF_binding_site

# Final reformating

  * gzip all CSV files
  * add extra formatting to yaml config file

In [7]:
!find ../../datasets/human_ocr_ensembl/ -type f -name "*.csv" -exec gzip {} \;

In [8]:
with open("../../datasets/human_ocr_ensembl/metadata.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz'}},
 'version': 0}

In [9]:
config['classes']['positive']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 
config['classes']['negative']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'}},
 'version': 0}

In [10]:
with open("../../datasets/human_ocr_ensembl/metadata.yaml", 'w') as handle:
  yaml.dump(config, handle)

# Testing

In [11]:
from genomic_benchmarks.loc2seq import download_dataset

download_dataset("human_ocr_ensembl")

/home/katarina/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz: 1.11GB [04:49, 3.82MB/s]                               
100%|██████████| 24/24 [00:30<00:00,  1.27s/it]


PosixPath('/home/katarina/.genomic_benchmarks/human_ocr_ensembl_regulatory')

In [12]:
from genomic_benchmarks.data_check import info

info("human_ocr_ensembl", 0)

Dataset `human_ocr_ensembl_regulatory` has 2 classes: negative, positive.

The lenght of genomic intervals ranges from 71 to 593, with average 326.3452470873675 and median 315.0.

Totally 174756 sequences have been found, 139804 for training and 34952 for testing.


,train,test
negative,69902,17476
positive,69902,17476
